1. How To load Dataset
2. How to token?
3. How to build vocabulary?
4. How to convert word to index
5. How to find word's index to reverse transform?

In [61]:
import torchtext.datasets as datasets
import torchtext.data as data

### 1. How To load Dataset && 2. How to token?
1. for translation task,you need pair of data(src,trg),
2. given file path and exts
3. given token,which mean you must tell your alg how to convert a string to batch of words

In [62]:
MAX=56
def my_custom_tokenizer(s):
    return s.split()
SRC= data.Field(tokenize=my_custom_tokenizer,init_token='<s>',eos_token='</s>',unk_token='<unk>',pad_token='<unk>')
TGT= data.Field(tokenize=my_custom_tokenizer,init_token='<s>',eos_token='</s>',unk_token='<unk>',pad_token='<unk>')
mt_train = datasets.TranslationDataset(
    path='.data/iwslt/de-en/IWSLT16.TED.dev2010.de-en', 
    exts=('.en','.de'),
    fields=(SRC, TGT),
    filter_pred=lambda x:len(x.src)<MAX and len(x.trg)<MAX)
#when you get dataset,you can see info about your set
print('Examples:',len(mt_train))
#and see how your dataset look like
head=10
for i ,ex in enumerate(mt_train):
    print(ex.src[:4],'--->',ex.trg[:4])
    if i> head:break
print(len(mt_train.examples[0].src))
print(len(mt_train.examples[0].trg))

Examples: 859
['You', 'know,', 'one', 'of'] ---> ['Wissen', 'Sie,', 'eines', 'der']
['And', 'of', 'course,', 'we'] ---> ['Und', 'natürlich', 'teilen', 'wir']
["We're", 'all', 'born.', 'We'] ---> ['Wir', 'werden', 'alle', 'geboren.']
['We', 'go', 'through', 'initiation'] ---> ['Wir', 'durchlaufen', 'Initiationsrituale.']
['We', 'have', 'to', 'deal'] ---> ['Wir', 'müssen', 'uns', 'mit']
['But', "what's", 'interesting', 'is'] ---> ['Aber', 'interessant', 'ist', 'der']
['All', 'of', 'these', 'peoples'] ---> ['All', 'diese', 'Menschen', 'lehren']
['And', 'this', 'is', 'an'] ---> ['Und', 'das', 'ist', 'eine']
['Now,', 'together', 'the', 'myriad'] ---> ['Zusammen', 'bilden', 'die', 'unzähligen']
['And', 'you', 'might', 'think'] ---> ['Man', 'kann', 'sich', 'dieses']
['The', 'ethnosphere', 'is', "humanity's"] ---> ['Die', 'Ethnosphäre', 'ist', 'das']
["It's", 'the', 'symbol', 'of'] ---> ['Sie', 'ist', 'das', 'Symbol']
54
55


### How to build vocabulary? &&How to convert word to index

In [63]:
SRC.build_vocab(mt_train.src)
print('num of words:',len(SRC.vocab.itos))
TGT.build_vocab(mt_train.trg)
print('num of words:',len(TGT.vocab.itos))


num of words: 3933
num of words: 4613


### How To Batch?

In [97]:
#once src's word or tgt's words >12000,then stop prefench data into queue
#just using that data for your training!
global maxSeqLen
BATCH=12000
def batch_size_fn(new,count,validbatch):
    global maxSeqLen
    if count==1:maxSeqLen=0
    maxSeqLen=max(maxSeqLen,len(new.src),len(new.trg))
    return maxSeqLen*count
gen=data.batch(mt_train,BATCH,batch_size_fn=batch_size_fn)
for x in gen:
    print(len(x))

218
218
218
205


### How To convert to tensor

In [102]:
#以下函数解释:
# 默认批次=99,但是由于example的长短不一,
#如果这99个example src包含的字数<2.5W并且tgt包含的字数<2.5W,那么一批数量是99
#头在返回的批量<99,哪里

global maxSeqLen
BATCH=12000
def batch_size_fn(new,count,validbatch):
    global maxSeqLen
    if count==1:maxSeqLen=0
    maxSeqLen=max(maxSeqLen,len(new.src),len(new.trg))
    return maxSeqLen*count
gen=data.batch(mt_train,BATCH,batch_size_fn=batch_size_fn)

iters=data.Iterator(mt_train,
              batch_size=BATCH, 
              sort_key=lambda x:(len(x.src),len(x.trg)),
              train=True, 
              repeat=False, 
              shuffle=False,
              batch_size_fn=batch_size_fn)

def rebatch(data_iter):
    for d in data_iter:
        yield (d.src.transpose(0,1),d.trg.transpose(0,1))
for x,y in rebatch(iters):
    print(x.size(),y.size())
x,y=x[0].numpy(),y[0].numpy()
x=' '.join([SRC.vocab.itos[ii] for ii in x])
print(x)
print(y)
y=' '.join([TGT.vocab.itos[ii] for ii in y])
print(y)

torch.Size([218, 56]) torch.Size([218, 57])
torch.Size([218, 57]) torch.Size([218, 57])
torch.Size([218, 57]) torch.Size([218, 53])
torch.Size([205, 45]) torch.Size([205, 41])
<s> But then I remembered Tom talking to the open air and I tried it. </s> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>
[   1   53   80 1085   14   78 1055   24  331   10   22 3410  852  652
  114    5   14 4373 1099    2    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0]
<s> Aber dann erinnerte ich mich daran, wie Tom in den freien Himmel gesprochen hatte und ich versuchte es. </s> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>
